In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 30.8 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     |████████████████████████████████| 895 kB 75.3 MB/s 
     |████████████████████████████████| 6.6 MB 65.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
import os
from tqdm.notebook import tqdm
import glob
import sys
import time
import pickle
import cv2
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import random
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import get_scheduler

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
%cd /content/drive/MyDrive/DL_Project

/content/drive/MyDrive/DL_Project


In [17]:
BASE_DIR = './Dataset/'

In [98]:
class CaptchaDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(f"{root_dir}{annotation_file}", dtype=str)
        self.annotations = self.annotations[self.annotations['length']=='5']
        self.annotations = self.annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index]
        img = Image.open(
            os.path.join( self.root_dir, img_id['category'], f"Len{img_id['length']}Char", img_id['file'])
          ).convert("L")
        # print(img_id['length'], len(str(img_id['text'])))
        y_label = (int(img_id['length']), img_id['text']) #int(img_id['text'][1]), int(img_id['text'][2]), int(img_id['text'][3]), int(img_id['text'][4]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [99]:
transform = transforms.Compose(
    [
        # transforms.Resize((356, 356)),
        # transforms.RandomCrop((299, 299)),
        # transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, )),
    ]
)

In [100]:
BATCH = 16

In [101]:
dataset = CaptchaDataset(BASE_DIR, "Num.csv", transform = transform)
print(len(dataset))
train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [2686, 500, 500])
# train_set, validation_set = torch.utils.data.random_split(dataset, [8000,2001])

train_loader = DataLoader(dataset = train_set, shuffle = True, batch_size = BATCH)
validation_loader = DataLoader(dataset = validation_set, shuffle = True, batch_size = BATCH)
test_loader = DataLoader(dataset = test_set, shuffle = True, batch_size = BATCH)

3686


In [102]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(type(labels[1]))

<class 'torch.Tensor'>
torch.Size([16, 1, 60, 200])
<class 'tuple'>


In [23]:
DROPOUT_PROB = 0.25
# N_EPOCHS = 10
# model_path = './Models/'

In [109]:
class Model(nn.Module):
    def __init__(self, dropout_flag=1):
        super().__init__()

        # self.dropout_flag = dropout_flag  # 1->Conv Layer, 2->FC Layer

        self.blockA = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
        )

        self.blockA_drop = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(DROPOUT_PROB),
        )

        self.blockB = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
        )

        self.blockB_drop = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(DROPOUT_PROB),
        )

        self.blockC = nn.Sequential(
            nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
        )

        self.blockC_drop = nn.Sequential(
            nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(DROPOUT_PROB),
        )


        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(5376, 512)
        # self.fc2 = nn.Linear(512, 64)
        self.out1 = nn.Linear(512, 10)
        self.out2 = nn.Linear(512, 10)
        self.out3 = nn.Linear(512, 10)
        self.out4 = nn.Linear(512, 10)
        self.out5 = nn.Linear(512, 10)

        # self.length_out = nn.Linear(512, 5)

        self.softmax = nn.Softmax()

        # self.drop = nn.Dropout(DROPOUT_PROB)
        

    def forward(self, x):
        
        # if (self.dropout_flag==0):
        x = self.blockA_drop(x)
        x = self.pool(x)
        x = self.blockB_drop(x)
        x = self.pool(x)
        x = self.blockC_drop(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        # print(x.shape)
        x = self.fc1(x)
        label1 = self.softmax(self.out1(x))
        label2 = self.softmax(self.out2(x))
        label3 = self.softmax(self.out3(x))
        label4 = self.softmax(self.out4(x))
        label5 = self.softmax(self.out5(x))

        # length = self.softmax(self.length_out(x))

        return {'label': torch.stack([label1, label2, label3, label4, label5], dim=1)}

In [130]:
model = Model().to(device)

In [131]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)


In [132]:
## create the default learning rate scheduler
num_epochs = 20
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", 
    optimizer=optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)

In [133]:
# torch.Tensor([1]).to(device)

In [134]:
from torch.nn.utils.rnn import pad_sequence
# def train(model, title, regularization = None):
    
progress_bar = tqdm(range(num_training_steps))
trainlosslist = []
validationlosslist = []

for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct1 = 0
    train_correct2 = 0
    train_correct3 = 0
    train_correct4 = 0
    train_correct5 = 0
    train_correct = 0
    train_total = 0

    validation_loss = 0.0
    validation_correct = 0
    validation_total = 0

    model.train()
    for i, data in enumerate(train_loader):
        inputs = data[0].to(device)
        labels = data[1]

        # length = data[1][0][0]
        # t = torch.zeros(1, 5).to(device)
        # t[length-1] = 1
        # length = t

        label = []
        
        for l in range(len(labels[0])):
            label.append([])
            length = labels[0][l]

            for j in range(int(length)):
                character = labels[1][l][j]
                output_array = [0]*10
                output_array[int(character)] = 1
                label[l].append(output_array)


        label = torch.Tensor(label).to(device)

        outputs = model(inputs.to(device))
        # outputs['label'] = outputs['label'].reshape(1,5,11)

        # label = label.reshape(5,1,11)


        # label2_hat = outputs['label2']
        # label3_hat = outputs['label3']         
        # label4_hat = outputs['label4']         
        # label5_hat = outputs['label5']  
        # print(label1_hat.get_device())
        # print(label1.get_device())

        # print('loss -  ', loss_label)
        # print('Output - ', outputs)
        # print('Label - ', label)

        # print(outputs['label'].shape)
        # print(label.shape)


        # calculate loss
        loss_label = criterion(outputs['label'], label)
        # loss_length = criterion(outputs['length'], length.squeeze())
        # loss2 = criterion(label2_hat, label2.to(device).squeeze().type(torch.LongTensor))
        # loss3 = criterion(label3_hat, label3.to(device).squeeze().type(torch.LongTensor))     
        # loss4 = criterion(label4_hat, label4.to(device).squeeze().type(torch.LongTensor))     
        # loss5 = criterion(label5_hat, label5.to(device).squeeze().type(torch.LongTensor))

        # loss = loss #+ loss2 + loss3 + loss4 + loss5
        loss = loss_label

        pred = torch.argmax(outputs['label'], 2)
        # print(pred)
        # print(pred.shape)
        # print(label)
        # print(label.shape)
        
        label = torch.argmax(label, axis=2)

        train_correct += (torch.sum(label==pred, axis=1)==5).nonzero().shape[0]
        # _, predicted1 = torch.argmax(label.data, 1)
        # _, predicted2 = torch.max(label2_hat.data, 1)
        # _, predicted3 = torch.max(label3_hat.data, 1)
        # _, predicted4 = torch.max(label4_hat.data, 1)
        # _, predicted5 = torch.max(label5_hat.data, 1)

        train_total += BATCH

        correct = torch.sum(label==pred, axis=0)
        
        # train_correct1 += (torch.sum(label==pred, axis=1)==1).nonzero().shape[0]
        # train_correct2 += (torch.sum(label==pred, axis=1)==2).nonzero().shape[0]
        # train_correct3 += (torch.sum(label==pred, axis=1)==3).nonzero().shape[0]
        # train_correct4 += (torch.sum(label==pred, axis=1)==4).nonzero().shape[0]
        # train_correct5 += (torch.sum(label==pred, axis=1)==5).nonzero().shape[0]

        train_correct1 += (correct[0])
        train_correct2 += (correct[1])
        train_correct3 += (correct[2])
        train_correct4 += (correct[3])
        train_correct5 += (correct[4])

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)
        # print statistics
        train_loss += loss.item()
        
        
    # for i, data in enumerate(validation_loader):
    #     model.eval()
    #     inputs, labels = data[0].to(device), data[1].to(device)
    #     labels = labels.to(torch.float32)
        
    #     outputs = model(inputs.to(device))
    #     outputs = torch.reshape(outputs, (outputs.shape[0],))
    #     loss = criterion(outputs, labels.to(device))
    #     validation_loss += loss.item()
        
        
    #     predicted = (outputs>0.5).float()
    #     validation_total += labels.size(0)
    #     validation_correct1 += (predicted == labels.to(device)).sum().item()
    #     validation_correct2 += (predicted == labels.to(device)).sum().item()
    #     validation_correct3 += (predicted == labels.to(device)).sum().item()
    #     validation_correct4 += (predicted == labels.to(device)).sum().item()
    #     validation_correct5 += (predicted == labels.to(device)).sum().item()


    epoch_train_loss = train_loss/len(train_loader)

    # epoch_validation_loss = validation_loss/len(validation_loader) 
    epoch_train_acc1 = (train_correct1/train_total)*100
    epoch_train_acc2 = (train_correct2/train_total)*100
    epoch_train_acc3 = (train_correct3/train_total)*100
    epoch_train_acc4 = (train_correct4/train_total)*100
    epoch_train_acc5 = (train_correct5/train_total)*100
    # epoch_validation_acc = (validation_correct1/validation_total)*100

    print("Epoch [%d/%d]  Training Loss: %.5f  Acc1: %.5f  Acc2: %.5f  Acc3: %.5f  Acc4: %.5f  Acc5: %.5f " %(epoch+1, num_epochs, epoch_train_loss, epoch_train_acc1, epoch_train_acc2, epoch_train_acc3, epoch_train_acc4, epoch_train_acc5))
    print(f"Epoch {epoch},  Loss : {epoch_train_loss},  Acc : {train_correct/train_total}")
    # print("Epoch [%d/%d]  Training Loss: %.5f  Training Acc: %.5f  Validation Loss: %.5f   Validation Accuracy: %.5f " %(epoch+1, N_EPOCHS, epoch_train_loss, epoch_train_acc, epoch_validation_loss, epoch_validation_acc))
    trainlosslist.append(epoch_train_loss)
    # validationlosslist.append(epoch_validation_loss)
    
    # if (epoch_validation_loss < min_validation_loss):
    #     min_validation_loss = epoch_validation_loss
    #     torch.save(model.state_dict(), str(model_path)+str(title)+'.pt')
    

print('Finished Training')

# plot(trainlosslist, validationlosslist, title)


  0%|          | 0/3360 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
 53%|█████▎    | 1783/3360 [0

Epoch [1/20]  Training Loss: 0.80634  Acc1: 10.00744  Acc2: 9.70982  Acc3: 11.42113  Acc4: 10.26786  Acc5: 9.85863 
Epoch 0,  Loss : 0.8063393153605007,  Acc : 0.0



 10%|█         | 339/3360 [00:15<02:14, 22.54it/s]

Epoch [2/20]  Training Loss: 0.81307  Acc1: 9.44941  Acc2: 10.52827  Acc3: 9.85863  Acc4: 9.74702  Acc5: 10.08185 
Epoch 1,  Loss : 0.8130662405774707,  Acc : 0.0



 15%|█▌        | 507/3360 [00:22<02:03, 23.09it/s]

...

Epoch [18/20]  Training Loss: 0.81146  Acc1: 10.15625  Acc2: 9.41220  Acc3: 10.19345  Acc4: 10.11905  Acc5: 9.37500 
Epoch 17,  Loss : 0.8114574462884948,  Acc : 0.0



 95%|█████████▌| 3195/3360 [02:21<00:07, 21.58it/s]

Epoch [19/20]  Training Loss: 0.81143  Acc1: 10.15625  Acc2: 9.41220  Acc3: 10.19345  Acc4: 10.11905  Acc5: 9.37500 
Epoch 18,  Loss : 0.8114332907966205,  Acc : 0.0



100%|██████████| 3360/3360 [02:28<00:00, 23.02it/s]

Epoch [20/20]  Training Loss: 0.81144  Acc1: 10.15625  Acc2: 9.41220  Acc3: 10.19345  Acc4: 10.11905  Acc5: 9.37500 
Epoch 19,  Loss : 0.8114415215594428,  Acc : 0.0
Finished Training


In [ ]:
# from tqdm.notebook import tqdm

In [115]:
val_correct1 = 0
val_correct2 = 0
val_correct3 = 0
val_correct4 = 0
val_correct5 = 0
val_total = 0
val_correct = 0
for i, data in tqdm(enumerate(validation_loader)):
    inputs = data[0].to(device)
    labels = data[1]

    label = []
    
    for l in range(len(labels[0])):
        label.append([])
        length = labels[0][l]

        for j in range(int(length)):
            character = labels[1][l][j]
            output_array = [0]*10
            output_array[int(character)] = 1
            label[l].append(output_array)


    label = torch.Tensor(label).to(device)

    outputs = model(inputs.to(device))
    pred = torch.argmax(outputs['label'], 2)
    
    label = torch.argmax(label, axis=2)

    val_correct += (torch.sum(label==pred, axis=1)==5).nonzero().shape[0]
    
    val_total += BATCH
    
    val_correct1 += (torch.sum(label==pred, axis=1)==1).nonzero().shape[0]
    val_correct2 += (torch.sum(label==pred, axis=1)==2).nonzero().shape[0]
    val_correct3 += (torch.sum(label==pred, axis=1)==3).nonzero().shape[0]
    val_correct4 += (torch.sum(label==pred, axis=1)==4).nonzero().shape[0]
    val_correct5 += (torch.sum(label==pred, axis=1)==5).nonzero().shape[0]

    # loss.backward()
    # optimizer.step()
    # lr_scheduler.step()
    # progress_bar.update(1)
    # print statistics
    # train_loss += loss.item()

0it [00:00, ?it/s]
100%|██████████| 3360/3360 [04:12<00:00, 22.66it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [116]:
print(val_correct1/val_total, "1")
print(val_correct2/val_total, "2")
print(val_correct3/val_total, "3")
print(val_correct4/val_total, "4")
print(val_correct5/val_total, "5")
print()

0.359375 1
0.0703125 2
0.0078125 3
0.0 4
0.0 5



In [119]:
a = torch.zeros(16,5,10)
for i in range(16):
    a[i][0][3]=1
    a[i][1][4]=1
    a[i][2][9]=1
    a[i][3][1]=1
    a[i][4][7]=1
b = torch.zeros(16,5,10)
for i in range(16):
    b[i][0][1]=1
    b[i][1][4]=1
    b[i][2][9]=1
    b[i][3][3]=1
    b[i][4][7]=1

In [120]:
a = torch.argmax(a, axis=2)
b = torch.argmax(b, axis=2)


In [121]:
b[0][2]=6

In [124]:
torch.sum(a==b, axis=0)

tensor([ 0, 16, 15,  0, 16])

In [53]:
torch.argmax(a, axis=2)

tensor([[3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7],
        [3, 4, 9, 1, 7]])

In [ ]:
for i, data in enumerate(validation_loader):
    model.eval()

    inputs = data[0].to(device)
    labels = data[1]

    label1 = labels[1].to(device)
    # label2 = labels[2].to(device)
    # label3 = labels[3].to(device)
    # label4 = labels[4].to(device)
    # label5 = labels[5].to(device)
    

    # forward + backward + optimize
    outputs = model(inputs.to(device))

    label1_hat = outputs['label1']
    # label2_hat = outputs['label2']
    # label3_hat = outputs['label3']         
    # label4_hat = outputs['label4']         
    # label5_hat = outputs['label5']  

    _, predicted1 = torch.max(label1_hat.data, 1)
    # _, predicted2 = torch.max(label2_hat.data, 1)
    # _, predicted3 = torch.max(label3_hat.data, 1)
    # _, predicted4 = torch.max(label4_hat.data, 1)
    # _, predicted5 = torch.max(label5_hat.data, 1)

    validation_total += label1.size(0)

    validation_correct1 += (predicted1 == label1.to(device)).sum().item()
    # validation_correct2 += (predicted2 == label2.to(device)).sum().item()
    # validation_correct3 += (predicted3 == label3.to(device)).sum().item()
    # validation_correct4 += (predicted4 == label4.to(device)).sum().item()
    # validation_correct5 += (predicted5 == label5.to(device)).sum().item()

    # epoch_validation_loss = validation_loss/len(validation_loader) 
    epoch_validation_acc1 = (validation_correct1/validation_total)*100
    # epoch_validation_acc2 = (validation_correct2/validation_total)*100
    # epoch_validation_acc3 = (validation_correct3/validation_total)*100
    # epoch_validation_acc4 = (validation_correct4/validation_total)*100
    # epoch_validation_acc5 = (validation_correct5/validation_total)*100

    epoch_validation_acc2 = 0
    epoch_validation_acc3 = 0
    epoch_validation_acc4 = 0
    epoch_validation_acc5 = 0

    print("Epoch [%d/%d]  Acc1: %.5f  Acc2: %.5f  Acc3: %.5f  Acc4: %.5f  Acc5: %.5f " %(epoch+1, num_epochs, epoch_validation_acc1, epoch_validation_acc2, epoch_validation_acc3, epoch_validation_acc4, epoch_validation_acc5))
    


NameError: ignored

In [ ]:
# model = Model().to(device)

In [ ]:
# title = 'yeah'
# train(model, title)

  0%|          | 0/10 [01:47<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
# def load_data(path, C_type='/AlphaNum', char_len = 5):
#     print('loading dataset')
#     X_train = []
#     Y_train = []
#     X_test = []
#     Y_test = []

#     path = path + C_type

#     for root, dirs, files in os.walk(path):
#         for f in files:
#             if 'Len5Char' in os.path.join(root,f):
#                 if '.png' in f:
#                     fl = f.split("_")[0]
#                     print(fl)
                    

#                     label = np.zeros((char_len, 36))
#                     for i in range(char_len):
#                         label[i, int(fl[i])] = 1

#                     img = cv2.imread(os.path.join(root, fl))
#                     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                     img = cv2.resize(img, (int(135/2), int(50/2)), interpolation=cv2.INTER_AREA)
#                     img = np.reshape(img, (img.shape[0], img.shape[1], 1))

#                     if random() < 0.1:
#                         Y_test.append(label)
#                         X_test.append(img)
#                     else:
#                         Y_train.append(label)
#                         X_train.append(img)






## Visualize Images

In [ ]:
def imshow(img, title):
    plt.figure()
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1,2,0))
    plt.title(title)
    plt.imshow(npimg)
    plt.show()

dataiter = iter(dataloader)
images, labels = dataiter.next()

c=0
for i in range(len(images)):
    if (labels[i]==0):
        c+=1
        imshow(images[i], "Parasitized")
    if (c==5):
        break

c=0
for i in range(len(images)):
    if (labels[i]==1):
        c+=1
        imshow(images[i], "Uninfected")
    if (c==5):
        break

In [ ]:
def plot(trainlosslist, validationlosslist, title):
    epochs = range(1,N_EPOCHS+1)
    plt.plot(epochs, trainlosslist, 'g', label='Training loss')
    plt.plot(epochs, validationlosslist, 'b', label='Validation loss')
    plt.title('Training and Validation loss  - ' + str(title))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def testing(model):
    
    model.eval()
    
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images.to(device))
            outputs = torch.reshape(outputs, (outputs.shape[0],))
            predicted = (outputs>0.5).float()
            test_total += labels.size(0)
            test_correct += (predicted == labels.to(device)).sum().item()
            

    print('Accuracy of the network on the test images: %.5f %%' % (100*test_correct/test_total))